# CAPSTONE
## Battle of the Neighbourhoods
### Location analysis for opening a New Restaurant in Chennai

Dhivya dharani
---------------------------------------------------------------------------------------------------
#### Introduction
This final project explores the best locations for opening a restaurant throughout the Chennai city Food Business is always the best one. Since there are a lot of Restaurants available, analysis is required to find out the affordable area and to find out available venues in the area. This report explores which neighbourhoods of Chennai and their land values per Sq. Ft and most common venues in each neighbourhood. Additionally, the report is grouped based on the area prices.
#### Data
In order to answer the above questions, data on Chennai City neighbourhoods, latitude, longitude, available venues are required.
Chennai City data containing the neighbourhoods and land prices are taken from https://www.livechennai.com/Propertyratesinchennai.asp
All data related to locations and quality of venues will be obtained via the Foursquare API utilized via the Request library in Python.
#### Methodology
• Data will be collected from https://www.livechennai.com/Propertyratesinchennai.asp and cleaned and processed into a data frame.
• Foursquare be used to locate all venues and then filtered by Italian restaurants. Ratings, tips, and likes by users will be counted and added to the data frame.
• Data will be sorted based on rankings
• Finally, the data be will be grouped into clusters based on the land prices.
#### Problem Statement
1.	What is / are the common venues and land prices in Chennai City?
2.	In what Neighbourhood should I open a restaurant to have the best chance of being successful?



In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # 
#import folium # map rendering library

#Lib for html handling
from lxml import html

print('Libraries imported.')


Libraries imported.


In [2]:

!conda install -c conda-forge folium=0.5.0 --yes # 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [3]:
import lxml.html as lh
url1 = 'https://www.livechennai.com/Propertyratesinchennai.asp'
pageContent=requests.get(url1)
price_html = lh.fromstring(pageContent.content)

In [4]:
tr_elements = price_html.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:10]]

[0, 1, 1, 1, 1, 1, 4, 4, 4, 4]

In [5]:
tr_elements = price_html.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[6]:
    i+=1
    name=t.text_content()
    print (i,name)
    col.append((name,[]))

1 Locality
2 Min
3 Max
4 Average Price


In [6]:
len(tr_elements)

57

In [7]:
for j in range(7,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    print(len(T))
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=4:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        print(data)
        #Increment i for the next column
        i+=1

4
Adyar
10,180
17,159
13,670	
4
Alwarpet
10,991
18,401
14,696	
4
Ambattur
4,176
5,936
5,056	
4
Ambattur Industrial Estate 
4,548
6,111
5,330
4
Anna Nagar
7,799
12,880
10,340
4
Anna Nagar West
6,350
10,554
8,452
4
Arcot Road
5,966
10,103
8,035
4
Avadi
3,271
4,427
3,849
4
Chromepet
3,933
6,058
4,996
4
East Coast Road
4,036
5,695
4,865
4
Egmore
11,851
17,358
14,604
4
G S T Road
3,235
5,040
4,137	
4
Guduvancheri
3,028
4,625
3,826
4
Kelambakkam
3,094
4,370
3,732
4
Kilpauk
7,168
12,200
9,684
4
Kodambakkam
5,424
9,460
7,442
4
Kolathur
4,402
6,098
5,250
4
Korattur
4,939
7,098
6,018
4
Koyambedu
7,202
10,023
8,612
4
Madipakkam
3,950
5,910
4,930
4
Mandaiveli
12,664
18,457
15,561
4
Medavakkam
3,851
5,606
4,729
4
Mogappair
5,056
7,452
6,254
4
Mogappair West
5,068
7,503
6,286
4
Mylapore
9,422
17,298
13,360
4
Navalur
3,272
5,368
4,320
4
Nungambakkam
11,906
21,676
16,791
4
OMR
3,474
5,728
4,601
4
Oragadam
3,041
4,642
3,841
4
Perungudi
5,821
8,079
6,950
4
Poonamallee
3,399
4,831
4,115
4
Porur
4,865
7,4

In [8]:
[len(C) for (title,C) in col]

[32, 32, 32, 32]

In [9]:
Dict={title:column for (title,column) in col}
df_price=pd.DataFrame(Dict)

In [10]:
df_price

,Locality,Min,Max,Average Price
0,Adyar,"10,180","17,159","13,670\t"
1,Alwarpet,"10,991","18,401","14,696\t"
2,Ambattur,"4,176","5,936","5,056\t"
3,Ambattur Industrial Estate,"4,548","6,111","5,330"
4,Anna Nagar,"7,799","12,880","10,340"
5,Anna Nagar West,"6,350","10,554","8,452"
6,Arcot Road,"5,966","10,103","8,035"
7,Avadi,"3,271","4,427","3,849"
8,Chromepet,"3,933","6,058","4,996"
9,East Coast Road,"4,036","5,695","4,865"


In [11]:

df_price["Min"]= df_price["Min"].str.replace(",", "", case = False) 
df_price["Max"]= df_price["Max"].str.replace(",", "", case = False) 
df_price["Average Price"]= df_price["Average Price"].str.replace("\t", "", case = False) 
df_price["Average Price"]= df_price["Average Price"].str.replace(",", "", case = False) 
df_price.head()

,Locality,Min,Max,Average Price
0,Adyar,10180,17159,13670
1,Alwarpet,10991,18401,14696
2,Ambattur,4176,5936,5056
3,Ambattur Industrial Estate,4548,6111,5330
4,Anna Nagar,7799,12880,10340


In [12]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [13]:
for index, item in df_price.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Locality}")

index: 0
item: Locality         Adyar
Min              10180
Max              17159
Average Price    13670
Name: 0, dtype: object
item.Street only: Adyar
index: 1
item: Locality         Alwarpet
Min                 10991
Max                 18401
Average Price       14696
Name: 1, dtype: object
item.Street only: Alwarpet
index: 2
item: Locality         Ambattur
Min                  4176
Max                  5936
Average Price        5056
Name: 2, dtype: object
item.Street only: Ambattur
index: 3
item: Locality         Ambattur Industrial Estate 
Min                                     4548
Max                                     6111
Average Price                           5330
Name: 3, dtype: object
item.Street only: Ambattur Industrial Estate 
index: 4
item: Locality         Anna Nagar
Min                    7799
Max                   12880
Average Price         10340
Name: 4, dtype: object
item.Street only: Anna Nagar
index: 5
item: Locality         Anna Nagar West
Min              

In [14]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [16]:
df_price['city_co_ord'] = df_price['Locality'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [17]:
df_price

,Locality,Min,Max,Average Price,city_co_ord
0,Adyar,10180,17159,13670,"(13.00645, 80.2577791)"
1,Alwarpet,10991,18401,14696,"(13.0338602, 80.2545491)"
2,Ambattur,4176,5936,5056,"(13.1128863, 80.15986241084877)"
3,Ambattur Industrial Estate,4548,6111,5330,"(13.0922855, 80.1675492)"
4,Anna Nagar,7799,12880,10340,"(13.0872004, 80.2164421)"
5,Anna Nagar West,6350,10554,8452,"(13.0940791, 80.1922978)"
6,Arcot Road,5966,10103,8035,"(52.4415218, -1.8458531)"
7,Avadi,3271,4427,3849,"(13.1254758, 80.09409)"
8,Chromepet,3933,6058,4996,"(12.9521897, 80.1411677)"
9,East Coast Road,4036,5695,4865,"(53.3948055, -1.4416333)"


In [18]:
df_price[['Latitude', 'Longitude']] = df_price['city_co_ord'].apply(pd.Series)

In [19]:
df_price.head()

,Locality,Min,Max,Average Price,city_co_ord,Latitude,Longitude
0,Adyar,10180,17159,13670,"(13.00645, 80.2577791)",13.006450,80.257779
1,Alwarpet,10991,18401,14696,"(13.0338602, 80.2545491)",13.033860,80.254549
2,Ambattur,4176,5936,5056,"(13.1128863, 80.15986241084877)",13.112886,80.159862
3,Ambattur Industrial Estate,4548,6111,5330,"(13.0922855, 80.1675492)",13.092285,80.167549
4,Anna Nagar,7799,12880,10340,"(13.0872004, 80.2164421)",13.087200,80.216442


In [20]:
df = df_price.drop(columns=['city_co_ord'])
df.head()

,Locality,Min,Max,Average Price,Latitude,Longitude
0,Adyar,10180,17159,13670,13.006450,80.257779
1,Alwarpet,10991,18401,14696,13.033860,80.254549
2,Ambattur,4176,5936,5056,13.112886,80.159862
3,Ambattur Industrial Estate,4548,6111,5330,13.092285,80.167549
4,Anna Nagar,7799,12880,10340,13.087200,80.216442


In [21]:
address = 'Chennai, IN'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of London City are 13.0801721, 80.2838331.


In [22]:
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Average Price'], df['Locality']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai)  
    
map_chennai

In [23]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'H3VKU3AWSGY4ITJIHY3WRGZPVM2SS1R42W2WBAGAEL3SKCWM' # Foursquare ID
CLIENT_SECRET = 'VD0X1N12MSC4HKBJU31VHMTPRBYOA4CGLV0XBJUA3ZI5BIP2' # Foursquare Secret
VERSION = '20201201' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H3VKU3AWSGY4ITJIHY3WRGZPVM2SS1R42W2WBAGAEL3SKCWM
CLIENT_SECRET:VD0X1N12MSC4HKBJU31VHMTPRBYOA4CGLV0XBJUA3ZI5BIP2


In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Locality'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Adyar
Alwarpet
Ambattur
Ambattur Industrial Estate 
Anna Nagar
Anna Nagar West
Arcot Road
Avadi
Chromepet
East Coast Road
Egmore
G S T Road
Guduvancheri
Kelambakkam
Kilpauk
Kodambakkam
Kolathur
Korattur
Koyambedu
Madipakkam
Mandaiveli
Medavakkam
Mogappair
Mogappair West
Mylapore
Navalur
Nungambakkam
OMR
Oragadam
Perungudi
Poonamallee
Porur


In [29]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adyar,13.006450,80.257779,Bombay Brassiere,13.006961,80.256419,North Indian Restaurant
1,Adyar,13.006450,80.257779,Cool Biz,13.006632,80.257275,Juice Bar
2,Adyar,13.006450,80.257779,Prems Graama Bhojanam,13.006345,80.253995,Vegetarian / Vegan Restaurant
3,Adyar,13.006450,80.257779,Adyar Ananda Bhavan,13.005824,80.257368,Indian Restaurant
4,Adyar,13.006450,80.257779,Aavin Milk Booth,13.007646,80.259153,Grocery Store
5,Adyar,13.006450,80.257779,McDonald's,13.006632,80.254255,Fast Food Restaurant
6,Adyar,13.006450,80.257779,Rainforest Restaurant,13.006993,80.257767,Indian Restaurant
7,Adyar,13.006450,80.257779,Odyssey Book Shop,13.006682,80.253469,Bookstore
8,Adyar,13.006450,80.257779,M'bessy,13.007361,80.259560,Lounge
9,Adyar,13.006450,80.257779,Hotel Coronet,13.006548,80.257307,Indian Restaurant


In [30]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
Adyar,30,30,30,30,30,30
Alwarpet,24,24,24,24,24,24
Ambattur,3,3,3,3,3,3
Ambattur Industrial Estate,4,4,4,4,4,4
Anna Nagar,23,23,23,23,23,23
Anna Nagar West,5,5,5,5,5,5
Arcot Road,6,6,6,6,6,6
Avadi,3,3,3,3,3,3
Chromepet,11,11,11,11,11,11


In [31]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 77 uniques categories.


In [32]:

location_venues.shape

(226, 7)

In [33]:

# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Andhra Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Big Box Store,Bistro,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Daycare,Department Store,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gourmet Shop,Grocery Store,Gym,Hardware Store,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Lounge,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,North Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Pub,Rental Service,Restaurant,River,Road,Rock Club,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Tea Room,Thai Restaurant,Theater,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Wine Shop
0,Adyar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adyar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Adyar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Adyar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Adyar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
chennai_grouped = venues_onehot.groupby('Street').mean().reset_index()
chennai_grouped

,Street,Andhra Restaurant,Art Gallery,Asian Restaurant,Bakery,Bar,Big Box Store,Bistro,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Cafeteria,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Daycare,Department Store,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gourmet Shop,Grocery Store,Gym,Hardware Store,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Light Rail Station,Lounge,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,North Indian Restaurant,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Pub,Rental Service,Restaurant,River,Road,Rock Club,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,South Indian Restaurant,Tea Room,Thai Restaurant,Theater,Track Stadium,Train Station,Vegetarian / Vegan Restaurant,Wine Shop
0,Adyar,0.0,0.000000,0.033333,0.033333,0.00,0.000000,0.000000,0.033333,0.000000,0.00,0.0,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.00,0.00,0.000000,0.000000,0.033333,0.066667,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.300000,0.033333,0.000,0.033333,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00
1,Alwarpet,0.0,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.041667,0.041667,0.00,0.0,0.000000,0.000000,0.083333,0.041667,0.000000,0.041667,0.00,0.00,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.041667,0.125,0.041667,0.000000,0.000000,0.041667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.083333,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.00
2,Ambattur,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Ambattur Industrial Estate,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Anna Nagar,0.0,0.000000,0.043478,0.043478,0.00,0.000000,0.043478,0.043478,0.000000,0.00,0.0,0.000000,0.000000,0.086957,0.086957,0.043478,0.000000,0.00,0.00,0.000000,0.043478,0.000000,0.086957,0.000000,0.086957,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [35]:
chennai_grouped.shape

(29, 78)

In [36]:
num_top_venues = 5

for hood in chennai_grouped['Street']:
    print("----"+hood+"----")
    temp = chennai_grouped[chennai_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adyar----
                     venue  freq
0        Indian Restaurant  0.30
1        Electronics Store  0.07
2  North Indian Restaurant  0.07
3                     Café  0.03
4                Juice Bar  0.03


----Alwarpet----
                 venue  freq
0               Bakery  0.12
1  Japanese Restaurant  0.12
2           Restaurant  0.08
3                 Café  0.08
4   Chinese Restaurant  0.04


----Ambattur----
               venue  freq
0              River  0.33
1          Multiplex  0.33
2      Movie Theater  0.33
3  Andhra Restaurant  0.00
4              Motel  0.00


----Ambattur Industrial Estate ----
                  venue  freq
0           Pizza Place  0.25
1  Fast Food Restaurant  0.25
2            Food Court  0.25
3   Fried Chicken Joint  0.25
4                  Park  0.00


----Anna Nagar----
                  venue  freq
0     Indian Restaurant  0.13
1     Electronics Store  0.09
2  Fast Food Restaurant  0.09
3    Chinese Restaurant  0.09
4                  Café  

In [37]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [39]:
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = chennai_grouped['Street']

for ind in np.arange(chennai_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(chennai_grouped.iloc[ind, :], num_top_venues)

In [40]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adyar,Indian Restaurant,Electronics Store,North Indian Restaurant,Ice Cream Shop,Rock Club,Fast Food Restaurant,Coffee Shop,Lounge,Café,Juice Bar
1,Alwarpet,Bakery,Japanese Restaurant,Café,Restaurant,Sandwich Place,Gym,Juice Bar,Coffee Shop,Breakfast Spot,Bookstore
2,Ambattur,Multiplex,Movie Theater,River,Wine Shop,Daycare,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court
3,Ambattur Industrial Estate,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Food Court,Cosmetics Shop,Department Store,Dessert Shop,Electronics Store,Farmers Market,Food Truck
4,Anna Nagar,Indian Restaurant,Café,Fast Food Restaurant,Electronics Store,Chinese Restaurant,Paper / Office Supplies Store,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Department Store,Park


In [41]:
venues_sorted.shape


(29, 11)

In [42]:
chennai_grouped.shape

(29, 78)

In [43]:
chennai_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in Chennai, we could begin by clustering properties by venues/facilities/amenities nearby.

In [44]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

chennai_grouped_clustering = chennai_grouped.drop('Locality', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chennai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 1, 2, 2, 4, 4, 4, 2, 2, 2, 1, 2, 2, 2, 4, 0, 2, 0, 4, 2, 1, 2,
       0, 0, 1, 2, 3, 2, 2, 0, 2, 0], dtype=int32)

In [45]:
#Dataframe to include Clusters

chennai_grouped_clustering=df
chennai_grouped_clustering

,Locality,Min,Max,Average Price,Latitude,Longitude
0,Adyar,10180,17159,13670,13.006450,80.257779
1,Alwarpet,10991,18401,14696,13.033860,80.254549
2,Ambattur,4176,5936,5056,13.112886,80.159862
3,Ambattur Industrial Estate,4548,6111,5330,13.092285,80.167549
4,Anna Nagar,7799,12880,10340,13.087200,80.216442
5,Anna Nagar West,6350,10554,8452,13.094079,80.192298
6,Arcot Road,5966,10103,8035,52.441522,-1.845853
7,Avadi,3271,4427,3849,13.125476,80.094090
8,Chromepet,3933,6058,4996,12.952190,80.141168
9,East Coast Road,4036,5695,4865,53.394805,-1.441633


In [46]:
chennai_grouped_clustering.shape

(32, 6)

In [47]:
df.shape

(32, 6)

In [48]:
chennai_grouped_clustering.dtypes

Locality          object
Min               object
Max               object
Average Price     object
Latitude         float64
Longitude        float64
dtype: object

In [49]:
df.dtypes

Locality          object
Min               object
Max               object
Average Price     object
Latitude         float64
Longitude        float64
dtype: object

In [50]:
chennai_grouped_clustering['Cluster Labels'] = kmeans.labels_

In [51]:

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
chennai_grouped_clustering = chennai_grouped_clustering.join(venues_sorted.set_index('Street'), on='Locality')

In [52]:
chennai_grouped_clustering.head(30) # check the last columns!

,Locality,Min,Max,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adyar,10180,17159,13670,13.006450,80.257779,1,Indian Restaurant,Electronics Store,North Indian Restaurant,Ice Cream Shop,Rock Club,Fast Food Restaurant,Coffee Shop,Lounge,Café,Juice Bar
1,Alwarpet,10991,18401,14696,13.033860,80.254549,1,Bakery,Japanese Restaurant,Café,Restaurant,Sandwich Place,Gym,Juice Bar,Coffee Shop,Breakfast Spot,Bookstore
2,Ambattur,4176,5936,5056,13.112886,80.159862,2,Multiplex,Movie Theater,River,Wine Shop,Daycare,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court
3,Ambattur Industrial Estate,4548,6111,5330,13.092285,80.167549,2,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Food Court,Cosmetics Shop,Department Store,Dessert Shop,Electronics Store,Farmers Market,Food Truck
4,Anna Nagar,7799,12880,10340,13.087200,80.216442,4,Indian Restaurant,Café,Fast Food Restaurant,Electronics Store,Chinese Restaurant,Paper / Office Supplies Store,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Department Store,Park
5,Anna Nagar West,6350,10554,8452,13.094079,80.192298,4,Café,Ice Cream Shop,Juice Bar,Park,Food Truck,Department Store,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant
6,Arcot Road,5966,10103,8035,52.441522,-1.845853,4,Grocery Store,Track Stadium,Indian Restaurant,Hardware Store,Pizza Place,Wine Shop,Food Court,Department Store,Dessert Shop,Electronics Store
7,Avadi,3271,4427,3849,13.125476,80.094090,2,Art Gallery,IT Services,Rental Service,Wine Shop,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court
8,Chromepet,3933,6058,4996,12.952190,80.141168,2,Indian Restaurant,Ice Cream Shop,Shopping Mall,Movie Theater,Clothing Store,Bus Station,Road,Light Rail Station,Big Box Store,Bakery
9,East Coast Road,4036,5695,4865,53.394805,-1.441633,2,Wine Shop,Breakfast Spot,Coffee Shop,Brewery,Hardware Store,Gym,Grocery Store,Gourmet Shop,Furniture / Home Store,Hotel


In [53]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_grouped_clustering['Latitude'], chennai_grouped_clustering['Longitude'], chennai_grouped_clustering['Locality'], chennai_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
chennai_grouped_clustering.loc[chennai_grouped_clustering['Cluster Labels'] == 0, chennai_grouped_clustering.columns[[0] + list(range(3, chennai_grouped_clustering.shape[1]))]].head()

,Locality,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Kodambakkam,7442,13.049207,80.224283,0,Andhra Restaurant,Indian Restaurant,Coffee Shop,Grocery Store,Park,Food Court,Department Store,Dessert Shop,Electronics Store,Farmers Market
17,Korattur,6018,13.111539,80.184277,0,Fried Chicken Joint,Indian Restaurant,Convenience Store,Dessert Shop,Wine Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck
22,Mogappair,6254,13.083514,80.184078,0,Department Store,Vegetarian / Vegan Restaurant,Indian Restaurant,Bus Station,Sandwich Place,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant
23,Mogappair West,6286,13.082535,80.170917,0,Indian Restaurant,Electronics Store,Chinese Restaurant,Fast Food Restaurant,Wine Shop,Fried Chicken Joint,Dessert Shop,Farmers Market,Food Court,Food Truck
29,Perungudi,6950,12.971024,80.241805,0,Indian Restaurant,Pizza Place,Juice Bar,Fast Food Restaurant,Food Court,Café,Cafeteria,Wine Shop,Food Truck,Electronics Store


In [62]:
chennai_grouped_clustering.loc[chennai_grouped_clustering['Cluster Labels'] == 1, chennai_grouped_clustering.columns[[0] + list(range(3, chennai_grouped_clustering.shape[1]))]]

,Locality,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adyar,13670,13.006450,80.257779,1,Indian Restaurant,Electronics Store,North Indian Restaurant,Ice Cream Shop,Rock Club,Fast Food Restaurant,Coffee Shop,Lounge,Café,Juice Bar
1,Alwarpet,14696,13.033860,80.254549,1,Bakery,Japanese Restaurant,Café,Restaurant,Sandwich Place,Gym,Juice Bar,Coffee Shop,Breakfast Spot,Bookstore
10,Egmore,14604,13.072832,80.257691,1,Indian Restaurant,Farmers Market,Museum,Coffee Shop,Theater,Thai Restaurant,Motel,Department Store,Shopping Mall,Seafood Restaurant
20,Mandaiveli,15561,13.029206,80.266897,1,Cosmetics Shop,Hotel,Indian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Furniture / Home Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck
24,Mylapore,13360,13.031650,80.270099,1,Vegetarian / Vegan Restaurant,Indian Restaurant,Hotel,Juice Bar,Wine Shop,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant


In [64]:
chennai_grouped_clustering.loc[chennai_grouped_clustering['Cluster Labels'] == 2, chennai_grouped_clustering.columns[[0] + list(range(3, chennai_grouped_clustering.shape[1]))]]

,Locality,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ambattur,5056,13.112886,80.159862,2,Multiplex,Movie Theater,River,Wine Shop,Daycare,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court
3,Ambattur Industrial Estate,5330,13.092285,80.167549,2,Fried Chicken Joint,Pizza Place,Fast Food Restaurant,Food Court,Cosmetics Shop,Department Store,Dessert Shop,Electronics Store,Farmers Market,Food Truck
7,Avadi,3849,13.125476,80.094090,2,Art Gallery,IT Services,Rental Service,Wine Shop,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court
8,Chromepet,4996,12.952190,80.141168,2,Indian Restaurant,Ice Cream Shop,Shopping Mall,Movie Theater,Clothing Store,Bus Station,Road,Light Rail Station,Big Box Store,Bakery
9,East Coast Road,4865,53.394805,-1.441633,2,Wine Shop,Breakfast Spot,Coffee Shop,Brewery,Hardware Store,Gym,Grocery Store,Gourmet Shop,Furniture / Home Store,Hotel
11,G S T Road,4137,35.472958,-85.411651,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Guduvancheri,3826,12.843995,80.060834,2,Indian Restaurant,Department Store,Train Station,Bakery,Bus Station,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant
13,Kelambakkam,3732,12.787144,80.219987,2,Bus Station,Indian Restaurant,Bar,Coffee Shop,Wine Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck
16,Kolathur,5250,13.124113,80.204628,2,Indian Restaurant,Pharmacy,Department Store,Restaurant,Wine Shop,Food Truck,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant
19,Madipakkam,4930,12.961135,80.200129,2,Grocery Store,Burger Joint,Wine Shop,Fried Chicken Joint,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court,Food Truck


In [60]:
chennai_grouped_clustering.loc[chennai_grouped_clustering['Cluster Labels'] == 3, chennai_grouped_clustering.columns[[0] + list(range(3, chennai_grouped_clustering.shape[1]))]]

,Locality,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Nungambakkam,16791,13.062063,80.240487,3,Indian Restaurant,Asian Restaurant,Café,Multicuisine Indian Restaurant,Bakery,South Indian Restaurant,Gourmet Shop,Juice Bar,Mexican Restaurant,Chinese Restaurant


In [59]:
chennai_grouped_clustering.loc[chennai_grouped_clustering['Cluster Labels'] == 4, chennai_grouped_clustering.columns[[0] + list(range(3, chennai_grouped_clustering.shape[1]))]]

,Locality,Average Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Anna Nagar,10340,13.087200,80.216442,4,Indian Restaurant,Café,Fast Food Restaurant,Electronics Store,Chinese Restaurant,Paper / Office Supplies Store,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Department Store,Park
5,Anna Nagar West,8452,13.094079,80.192298,4,Café,Ice Cream Shop,Juice Bar,Park,Food Truck,Department Store,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant
6,Arcot Road,8035,52.441522,-1.845853,4,Grocery Store,Track Stadium,Indian Restaurant,Hardware Store,Pizza Place,Wine Shop,Food Court,Department Store,Dessert Shop,Electronics Store
14,Kilpauk,9684,13.083215,80.237986,4,Frozen Yogurt Shop,Department Store,Coffee Shop,Fast Food Restaurant,Café,Food Truck,Daycare,Juice Bar,Indian Restaurant,Grocery Store
18,Koyambedu,8612,13.073450,80.194836,4,Metro Station,Pub,Bar,Movie Theater,Wine Shop,Dessert Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Food Court


#### Results:
As a result of our analysis with 31 we got 5 clusters based on the price. In that the second cluster has growing areas with affordable land rates. In which few neighbourhoods like Ambattur doesn’t have Indian Restaurant in most common venues. So that would be good choice for opening Indian Restaurant.
#### Conclusion:
Based on the analysis we found a group of neighbourhoods which can be considered for opening a restaurant. Since we have listed the most common venues, we could see few areas where there is no Indian restaurant, which could be idle for opening a new restaurant.

